In [8]:
# 导入各类包
import cv2 
import numpy as np 
import cv2 as cv 
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions 
from sklearn.metrics.pairwise import cosine_similarity 
import numpy as np 
from keras.preprocessing import image as Image
import ssl
import certifi

# Use certifi's path to the CA bundle as your default CA file in SSL
ssl_context = ssl.create_default_context(cafile=certifi.where())



In [9]:
# 定义函数计算图像的Hu矩
def calculate_hu_moments(image):
    # 转换为灰度图
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # 二值化
    ret, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
    # 计算Hu矩
    moments = cv2.moments(binary)
    hu_moments = cv2.HuMoments(moments)
    # 归一化处理
    hu_moments = -np.sign(hu_moments) * np.log10(np.abs(hu_moments) + np.finfo(float).eps)
    return hu_moments.flatten()

def extract_grid_image(image, position):
    # 假设九宫格每个模块的大小是原图像的三分之一
    height, width = image.shape[:2]
    grid_height = height // 3
    grid_width = width // 3

    # 根据位置提取模块图像
    x, y = position
    start_x = x * grid_width
    start_y = y * grid_height
    end_x = start_x + grid_width
    end_y = start_y + grid_height

    # 确保提取范围在图像尺寸之内
    end_x = min(end_x, width)
    end_y = min(end_y, height)

    # 提取模块图像
    grid_image = image[start_y:end_y, start_x:end_x]
    return grid_image

# 函数定义：从原始图像中提取指定位置的模块图像
def extract_grid_image(image, position):
    x, y = position
    height = image.shape[0] // 3
    width = image.shape[1] // 3
    return image[y * height:(y + 1) * height, x * width:(x + 1) * width]


骨架提取及评价


In [10]:
# 定义方法
def VThin(image, array):
    h = image.shape[0]
    w = image.shape[1]
    NEXT = 1
    for i in range(h):
        for j in range(w):
            if NEXT == 0:
                NEXT = 1
            else:
                M = image[i, j - 1] + image[i, j] + image[i, j + 1] if 0 < j < w - 1 else 1
                if image[i, j] == 0 and M != 0:
                    a = [0] * 9
                    for k in range(3):
                        for l in range(3):
                            if -1 < (i - 1 + k) < h and -1 < (j - 1 + l) < w and image[i - 1 + k, j - 1 + l] == 255:
                                a[k * 3 + l] = 1
                    sum = a[0] * 1 + a[1] * 2 + a[2] * 4 + a[3] * 8 + a[5] * 16 + a[6] * 32 + a[7] * 64 + a[8] * 128
                    image[i, j] = array[sum] * 255
                    if array[sum] == 1:
                        NEXT = 0
    return image

def HThin(image, array):
    h = image.shape[0]
    w = image.shape[1]
    NEXT = 1
    for j in range(h):
        for i in range(w):
            if NEXT == 0:
                NEXT = 1
            else:
                M = image[i - 1, j] + image[i, j] + image[i + 1, j] if 0 < i < h - 1 else 1
                if image[i, j] == 0 and M != 0:
                    a = [0] * 9
                    for k in range(3):
                        for l in range(3):
                            if -1 < (i - 1 + k) < h and -1 < (j - 1 + l) < w and image[i - 1 + k, j - 1 + l] == 255:
                                a[k * 3 + l] = 1
                    sum = a[0] * 1 + a[1] * 2 + a[2] * 4 + a[3] * 8 + a[5] * 16 + a[6] * 32 + a[7] * 64 + a[8] * 128
                    image[i, j] = array[sum] * 255
                    if array[sum] == 1:
                        NEXT = 0
    return image

def Xihua(image, array, num=20):
    h = image.shape[0]
    w = image.shape[1]
    iXihua = np.zeros((w, h, 1), dtype=np.uint8)
    np.copyto(iXihua, image)
    for i in range(num):
        VThin(iXihua, array)
        HThin(iXihua, array)
    return iXihua

def Two(image):
    h = image.shape[0]
    w = image.shape[1]

    iTwo = np.zeros((w, h, 1), dtype=np.uint8)
    for i in range(h):
        for j in range(w):
            iTwo[i, j] = 0 if image[i, j] < 200 else 255
    return iTwo

# 加载图片并预处理
def load_and_process_image(img_path):
    img = Image.load_img(img_path, target_size=(224, 224))
    img_array = Image.img_to_array(img)
    img_tensor = np.expand_dims(img_array, axis=0)
    img_tensor = preprocess_input(img_tensor)
    return img_tensor

# 提取特征
def extract_features(model, img_path):
    img_tensor = load_and_process_image(img_path)
    features = model.predict(img_tensor)
    # 将特征扁平化为一维数组
    flattened_features = features.flatten()
    return flattened_features


数据处理

In [11]:
model = VGG16(weights='imagenet', include_top=False)

dataUrl = 'data'

# 图片路径
img_path1 = f'{dataUrl}/Template_pictures/1.png'
img_path2 = f'{dataUrl}/Handcopied_pictures/2.png'
img_path3 = f'{dataUrl}/Handcopied_pictures/3.png'

array = [0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, \
         1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, \
         0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, \
         1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, \
         1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, \
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, \
         1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, \
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, \
         0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, \
         1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, \
         0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, \
         1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, \
         1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, \
         1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, \
         1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, \
         1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0]

# 提取特征向量
vector1 = np.array(extract_features(model, img_path1)).reshape(1, 25088)
vector2 = np.array(extract_features(model, img_path3)).reshape(1, 25088)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 260ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step


In [ ]:



# vector1 = np.array(extract_features(model, img_path1))
# vector2 = extract_features(model, img_path3)
normalized_distance = cosine_similarity(vector1, vector2)
print(f"The Euclidean distance between vector1 and vector2 is: {normalized_distance}")


Image = cv.imread(img_path1, 0)
iTwo = Two(Image)
iThin = Xihua(iTwo, array)
# cv.imshow('image', image)
# cv.imshow('iTwo', iTwo)

image1 = cv.imread(img_path2, 0)
iTwo1 = Two(image1)
iThin1 = Xihua(iTwo1, array)


cv.imwrite(f'{dataUrl}/output/a.png',iThin)
cv.imwrite(f'{dataUrl}/output/b.png',iThin1)
cv.waitKey(0)

The Euclidean distance between vector1 and vector2 is: [[0.31124064]]
